# Model Training Patterns

## Stochastic Gradient Descent
- On large datasets stochastic gradient descent (SGD) is applied to mini-batches
- This is called stochastic SGD and extensions of SGD e.g. Adam, Adamgrad etc... are the de facto optimiser used in modern-day machine learning frameworks

- SGD requires training to take place iteratively on small batches therefore training happens in a loop
- SGD finds a minimum, but not a closed-form solution, and so we have to detect whether the model convergence has happened
- As a result, the error (called the loss) on the training dataset has to be monitored.
- Overfitting can happen if the model complexity is higher than can be affored by the size and coverage of the dataset
- It is difficult to know if the compexity is too high until we actually train the model on the dataset
- Therefore, evaluation needs to be done within the training loop and error metrics on a witheld split of the training data (validation set).
- Because training and validation datasets have been used i the training loop it is necessary to withhold yet another split of the training dataset called the test set.
- Metrics are reported on the test set

## Design Pattern 11: Useful Overfitting